<a href="https://colab.research.google.com/github/perinai/tracing-ai/blob/main/Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Cell 1: Setup (Corrected)
!pip install flask pandas scikit-learn joblib pyngrok

In [8]:
# Cell 2: Data Generation and Preprocessing (Corrected)

import pandas as pd
import numpy as np

# --- Data Generation Function ---
def generate_health_data(records=1000):
    # The 'blood_oxygen' line is corrected here
    data = {
        'timestamp': pd.to_datetime(np.arange(records) * 60, unit='s'),
        'heart_rate': np.random.randint(60, 100, records),
        'blood_oxygen': np.round(np.random.uniform(95, 100, records), 1), # CORRECTED
        'activity_level': np.random.choice(['resting', 'walking', 'running'], records, p=[0.6, 0.3, 0.1])
    }
    df = pd.DataFrame(data)
    # Inject Anomalies
    for _ in range(int(records * 0.05)):
        idx = np.random.randint(0, records)
        df.loc[idx, 'heart_rate'] = np.random.randint(110, 150)
    for _ in range(int(records * 0.05)):
        idx = np.random.randint(0, records)
        # The 'blood_oxygen' line is corrected here as well
        df.loc[idx, 'blood_oxygen'] = round(np.random.uniform(85, 94), 1) # CORRECTED

    return df

# --- Data Preprocessing Function ---
def preprocess_data(df):
    df = pd.get_dummies(df, columns=['activity_level'], drop_first=True)
    df['anomaly'] = ((df['heart_rate'] > 100) | (df['blood_oxygen'] < 95)).astype(int)
    return df

# --- Execute and Save ---
raw_data = generate_health_data()
processed_data = preprocess_data(raw_data)

# Save the data to a file in the Colab environment
processed_data.to_csv('health_data_processed.csv', index=False)

print("Data generation and preprocessing complete.")
print("Created 'health_data_processed.csv' successfully.")
print("\nProcessed Data Head:")
print(processed_data.head())

Data generation and preprocessing complete.
Created 'health_data_processed.csv' successfully.

Processed Data Head:
            timestamp  heart_rate  blood_oxygen  activity_level_running  \
0 1970-01-01 00:00:00          99          96.7                   False   
1 1970-01-01 00:01:00          67          95.6                   False   
2 1970-01-01 00:02:00          70          98.0                    True   
3 1970-01-01 00:03:00          77          99.7                   False   
4 1970-01-01 00:04:00          98          97.0                   False   

   activity_level_walking  anomaly  
0                   False        0  
1                    True        0  
2                   False        0  
3                   False        0  
4                    True        0  


In [9]:
# Cell 3: AI Model Training and Saving

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# 1. Load Data from the CSV we created
df = pd.read_csv('health_data_processed.csv')

# 2. Define Features (X) and Target (y)
features = [col for col in df.columns if col not in ['timestamp', 'anomaly']]
X = df[features]
y = df['anomaly']

# 3. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 4. Train the Model
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

# 5. Evaluate the Model
y_pred = model.predict(X_test)
print("\n--- Model Evaluation Report ---")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Anomaly']))

# 6. Save the Trained Model and Feature List
joblib.dump(model, 'health_monitor_model.pkl')
joblib.dump(features, 'model_features.pkl')

print("\nModel saved as 'health_monitor_model.pkl'")
print("Model features saved as 'model_features.pkl'")


--- Model Evaluation Report ---
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00       180
     Anomaly       1.00      1.00      1.00        20

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200


Model saved as 'health_monitor_model.pkl'
Model features saved as 'model_features.pkl'


2y8Or6Scruc6QeqpJqcZtKq4XAD_4Rkkjkjk9WdMyUsujecrN

In [10]:
# Cell 4: Build and Run the Web App with pyngrok (Corrected and More Robust)

import os
import threading
from flask import Flask, request, render_template
from pyngrok import ngrok
import pandas as pd
import joblib

# --- Create HTML files first (no changes here) ---
if not os.path.exists('templates'):
    os.makedirs('templates')

index_html = """
<!doctype html>
<html lang="en"><head><title>Health Monitor</title><style>body{font-family:sans-serif;margin:2em;background-color:#f4f4f9;}h1{color:#333;}form{display:flex;flex-direction:column;width:300px;padding:20px;background:white;border-radius:8px;box-shadow:0 4px 6px rgba(0,0,0,0.1);}label,input,select{margin-bottom:12px;font-size:1.1em;}input,select{padding:8px;border:1px solid #ccc;border-radius:4px;}input[type=submit]{background-color:#007BFF;color:white;padding:12px;border:none;cursor:pointer;font-weight:bold;}input[type=submit]:hover{background-color:#0056b3;}</style></head><body><h1>Enter Your Health Metrics</h1><form action="/predict" method="post"><label for="heart_rate">Heart Rate (bpm):</label><input type="number" id="heart_rate" name="heart_rate" required value="75"><label for="blood_oxygen">Blood Oxygen (%):</label><input type="number" step="0.1" id="blood_oxygen" name="blood_oxygen" required value="98"><label for="activity_level">Current Activity:</label><select id="activity_level" name="activity_level"><option value="resting">Resting</option><option value="walking">Walking</option><option value="running">Running</option></select><input type="submit" value="Analyze My Health"></form></body></html>
"""
with open("templates/index.html", "w") as f:
    f.write(index_html)

result_html = """
<!doctype html>
<html lang="en"><head><title>Health Result</title><style>body{font-family:sans-serif;margin:2em;background-color:#f4f4f9;text-align:center;}.result-box{padding:30px;margin:auto;max-width:500px;border-radius:10px;color:white;box-shadow:0 4px 6px rgba(0,0,0,0.1);}.normal{background-color:#28a745;}.anomaly{background-color:#dc3545;}a{display:inline-block;margin-top:20px;padding:10px 20px;background-color:#6c757d;color:white;text-decoration:none;border-radius:4px;}a:hover{background-color:#5a6268;}</style></head><body><h1>Analysis Result</h1><div class="result-box {{ status_class }}"><h2>Status: {{ status }}</h2><p>{{ recommendation }}</p></div><br><a href="/">Check Again</a></body></html>
"""
with open("templates/result.html", "w") as f:
    f.write(result_html)


# --- Define the Flask App ---
app = Flask(__name__)

# Load the trained model and features
model = joblib.load('health_monitor_model.pkl')
model_features = joblib.load('model_features.pkl')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    user_input = {
        'heart_rate': [float(request.form['heart_rate'])],
        'blood_oxygen': [float(request.form['blood_oxygen'])],
        'activity_level_running': [1 if request.form['activity_level'] == 'running' else 0],
        'activity_level_walking': [1 if request.form['activity_level'] == 'walking' else 0]
    }
    input_df = pd.DataFrame(user_input)
    final_df = input_df.reindex(columns=model_features, fill_value=0)

    prediction = model.predict(final_df)[0]
    proba = model.predict_proba(final_df)[0]

    if prediction == 1:
        status, status_class = "Anomaly Detected", "anomaly"
        recommendation = f"Warning: Metrics are unusual (Confidence: {proba[1]*100:.0f}%). Consider resting."
    else:
        status, status_class = "Normal", "normal"
        recommendation = f"Metrics appear normal (Confidence: {proba[0]*100:.0f}%). Keep it up!"

    return render_template('result.html', status=status, recommendation=recommendation, status_class=status_class)


# --- Setup ngrok and run the app ---
# Replace with your actual authtoken from the ngrok dashboard
NGROK_AUTHTOKEN = "2y8Or6Scruc6QeqpJqcZtKq4XAD_4Rkkjkjk9WdMyUsujecrN"

# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# Start ngrok tunnel and get the public URL
public_url = ngrok.connect(5000)
print(f" * Tunnel URL: {public_url}")

# Start the Flask app in a new thread
threading.Thread(target=app.run, kwargs={"host": "0.0.0.0", "port": 5000}).start()

 * Tunnel URL: NgrokTunnel: "https://7b07-34-66-26-193.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
